# Building Fast Queries on a CSV

In this project, we will imagine that we own an online laptop store and want to build a way to answer a frew different business questions about our inventory.

## The Dataset

We will use the `laptops.csv` file as our inventory. This CSV file was adapted from the [Laptop Prices dataset on Kaggle](https://www.kaggle.com/ionaskel/laptop-prices). I changed the first column name of the header to ID.

Here are the first few rows of the dataset:

In [1]:
import csv
with open('dataset/laptops.csv', encoding='latin-1') as open_file:
    read_file = csv.reader(open_file)
    rows = list(read_file)
    header = rows[0]
    rows = rows[1:]
    
print(header)
for row in rows[:5]:
    print(row)

['ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']
['1', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 2.3GHz', '8GB', '128GB SSD', 'Intel Iris Plus Graphics 640', 'macOS', '1.37kg', '1339.69']
['2', 'Apple', 'Macbook Air', 'Ultrabook', '13.3', '1440x900', 'Intel Core i5 1.8GHz', '8GB', '128GB Flash Storage', 'Intel HD Graphics 6000', 'macOS', '1.34kg', '898.94']
['3', 'HP', '250 G6', 'Notebook', '15.6', 'Full HD 1920x1080', 'Intel Core i5 7200U 2.5GHz', '8GB', '256GB SSD', 'Intel HD Graphics 620', 'No OS', '1.86kg', '575']
['4', 'Apple', 'MacBook Pro', 'Ultrabook', '15.4', 'IPS Panel Retina Display 2880x1800', 'Intel Core i7 2.7GHz', '16GB', '512GB SSD', 'AMD Radeon Pro 455', 'macOS', '1.83kg', '2537.45']
['5', 'Apple', 'MacBook Pro', 'Ultrabook', '13.3', 'IPS Panel Retina Display 2560x1600', 'Intel Core i5 3.1GHz', '8GB', '256GB SSD', 'Intel Iris Pl

Here is a brief description of the rows:
* ID: A unique identifier for the laptop.
* Company: The name of the company that produces the laptop.
* Product: The name of the laptop.
* TypeName: The type of laptop.
* Inches: The size of the screen in inches.
* ScreenResolution: The resolution of the screen.
* CPU: The laptop CPU
* RAM: The amount of RAM in the laptop.
* Memory: The size of the hard drive.
* GPU: The graphics card name.
* OpSys: The name of the operating system.
* Weight: The laptop weight.
* Price: The price of the laptop.

## Inventory Class

The goal of this guided project is to create a class that represents our inventory. The methods in that class will implement the queries that we want to answer about our inventory. We will also preprocess that data to make those queries run faster.

Here are some queries that we will want to answer:
* Given a laptop id, find the corresponding data.
* Given an amount of money, find whether there are two laptops whose total price is that given amount.
* Identify all laptops whose price falls within a given budget.

Let's start by implementing the constructor. It will take the name of the CSV file as argument and then read the rows contained in it.

In [2]:
class Inventory():
    def __init__(self, csv_filename):
        with open(''.join(['dataset/', csv_filename]), encoding='latin-1') as open_file:
            read_file = csv.reader(open_file)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1])

laptop_inven = Inventory('laptops.csv')
print('Header:', laptop_inven.header)
print('# of rows:', len(laptop_inven.rows))

Header: ['ID', 'Company', 'Product', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price_euros']
# of rows: 1303


## Finding a Laptop From the ID

The first thing that we will implement is a way to look up a laptop from a given identifier. In this way, when a customer to our store with a purchase slip, we can quickly identify the laptop to which it corresponds.

![nn](img/laptop_id_search.gif)

For this, we will write a function named `get_laptop_from_id()`. This function will take as argument the identifier of the laptop and return the full row of the laptop with that id. 

In [3]:
class Inventory():
    def __init__(self, csv_filename):
        with open(''.join(['dataset/', csv_filename]), encoding='latin-1') as open_file:
            read_file = csv.reader(open_file)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
laptop_inven = Inventory('laptops.csv')
print('Search 64:', laptop_inven.get_laptop_from_id('64'))
print('Search 65:', laptop_inven.get_laptop_from_id('65'))

Search 64: ['64', 'Asus', 'UX410UA-GV350T (i5-8250U/8GB/256GB/FHD/W10)', 'Notebook', '14', 'Full HD 1920x1080', 'Intel Core i5 8250U 1.6GHz', '8GB', '256GB SSD', 'Intel UHD Graphics 620', 'Windows 10', '1.4kg', 941.0]
Search 65: None


The algorithm requires us to look at every single row to find the one that we are looking for (or decide that such a row does not exist). This algorithm has time complexity O(R) where R is the number of rows.

## Improving ID Lookups

We can solve this problem more efficiently by preprocessing the data. The idea is to preprocess the data into a dictionary where the keys are the IDs and the values the rows. Then, we will use that dictionary in the `get_laptop_from_id()` method. We can do this by:
1. Preprocess the data and create the dictionary in the `__init__()` method.
2. Re-implement the `get_laptop_from_id()` method. We will do it as a new method so that we can compare the two.

In [4]:
class Inventory():
    def __init__(self, csv_filename):
        with open(''.join(['dataset/', csv_filename]), encoding='latin-1') as open_file:
            read_file = csv.reader(open_file)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
laptop_inven = Inventory('laptops.csv')
print('Search 64:', laptop_inven.get_laptop_from_id_fast('64'))
print('Search 65:', laptop_inven.get_laptop_from_id_fast('65'))

Search 64: ['64', 'Asus', 'UX410UA-GV350T (i5-8250U/8GB/256GB/FHD/W10)', 'Notebook', '14', 'Full HD 1920x1080', 'Intel Core i5 8250U 1.6GHz', '8GB', '256GB SSD', 'Intel UHD Graphics 620', 'Windows 10', '1.4kg', 941.0]
Search 65: None


## Comparing the Performance

The `get_laptop_from_id()` method has time complexity O(R) where R is the number of rows. In contrast, the new implementation as time complexity O(1). It does so by using more memory to store the self.id_to_row dictionary and using a bit more time creating an instance (because it needs to create the dictionary).

Let's experiment to compare the performance of the two methods. The idea is to generate random IDs using the `random` module. Then, use both methods to lookup these same IDs. We will use the `time` module to measure the execution time of each lookup and, for each method, add all times together.

In [5]:
import time, random

ids = [str(random.randint(1, 9999)) for _ in range(10000)]
laptop_inven = Inventory('laptops.csv')

total_time_no_dict = 0
for id in ids:
    start = time.time()
    laptop_inven.get_laptop_from_id(id)
    end = time.time()
    total_time_no_dict += (end - start)

total_time_dict = 0
for id in ids:
    start = time.time()
    laptop_inven.get_laptop_from_id_fast(id)
    end = time.time()
    total_time_dict += (end - start)
    
print('Total time of get_laptop_from_id():', total_time_no_dict)
print('Total time of get_laptop_from_id_fast():', total_time_dict)

Total time of get_laptop_from_id(): 1.2374043464660645
Total time of get_laptop_from_id_fast(): 0.008726835250854492


## Two Laptop Promotion

Sometimes, your store offers a promotion where you give a gift card. A customer can use the gift to buy up to two laptops. To avoid having to keep track of what was already spent, the gift card has a single time usage. This means that, even if there is leftover money, it cannot be used anymore. For example, imagine that there are only three laptops in inventory:

![nn](img/laptop_promotion.png)

The prices of these laptops are \\$1,339, \\$898, and \\$575. Say we offered a gift card of \\$2,500. Since a customer can buy, at most two laptops with a gift card, the maximum thay can spend is \\$2,237 (\\$1,339 plus \\$898). Therefore, they might feel cheated because no matter how they spend their gift card, they cannot spend the full \\$2,500.

You don't want to make a customer feel cheated, so whenever you issue a gift card, you want to make sure that there is at least one way to spend it in full. In other words, before issuing a gift card for D dollars, you want to make sure that either there is a laptop that costs exactly D dollars or two laptops whose costs add up to precisely D dollars.

In [6]:
class Inventory():
    def __init__(self, csv_filename):
        with open(''.join(['dataset/', csv_filename]), encoding='latin-1') as open_file:
            read_file = csv.reader(open_file)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            price = float(row[-1])
            if price == dollars:
                return True
        for i in range(len(self.rows)):
            for j in range(i, len(self.rows)):
                price1 = float(self.rows[i][-1])
                price2 = float(self.rows[j][-1])
                if price1 + price2 == dollars:
                    return True
        return False

laptop_inven = Inventory('laptops.csv')
print('$1000:', laptop_inven.check_promotion_dollars(1000))
print('$442:', laptop_inven.check_promotion_dollars(442))

$1000: True
$442: False


## Optimizing Laptop Promotion

We can preprocess data to make our code run faster.

Since we only care about whether or not there is a solution, we can store all laptops prices in a set when we initialize the inventory. Then we can check in constant time whether there is a laptop with a given price.

In [7]:
class Inventory():
    def __init__(self, csv_filename):
        with open(''.join(['dataset/', csv_filename]), encoding='latin-1') as open_file:
            read_file = csv.reader(open_file)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            price = float(row[-1])
            self.prices.add(price)
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            price = float(row[-1])
            if price == dollars:
                return True
        for i in range(len(self.rows)):
            for j in range(i, len(self.rows)):
                price1 = float(self.rows[i][-1])
                price2 = float(self.rows[j][-1])
                if price1 + price2 == dollars:
                    return True
        return False

    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price1 in self.prices:
            price2 = dollars - price1
            if price2 in self.prices:
                return True
        return False

laptop_inven = Inventory('laptops.csv')
print('$1000:', laptop_inven.check_promotion_dollars_fast(1000))
print('$442:', laptop_inven.check_promotion_dollars_fast(442))

$1000: True
$442: False


## Comparing Promotion Functions

Let's compare the performance of the last two functions that we wrote.

In [8]:
prices = [random.randint(100, 5000) for _ in range(100)]
laptop_inven = Inventory('laptops.csv')

total_time_no_set = 0
for price in prices:
    start = time.time()
    laptop_inven.check_promotion_dollars(price)
    end = time.time()
    total_time_no_set += (end - start)

total_time_set = 0
for price in prices:
    start = time.time()
    laptop_inven.check_promotion_dollars_fast(price)
    end = time.time()
    total_time_set += (end - start)
    
print('Total time of check_promotion_dollars():', total_time_no_set)
print('Total time of check_promotion_dollars_fast():', total_time_set)

Total time of check_promotion_dollars(): 8.944240093231201
Total time of check_promotion_dollars_fast(): 0.003693103790283203


## Finding Laptops Within a Budget

We are going to leverage and extend binary search algorithm to help a customer find all laptops that fall within their budget.

More formally, we want to write a method that efficiently answers the query: Given a budget of D dollars, find all laptops whose price is at most D.

If we sort all laptops by price, we can use binary search to identify the first laptop in the sorted list within a price larger than D. We need to make sure that our binary search finds the first one on the list. Then, the result of the query will consist of all laptops whose index in the sorted list is smaller than the index of the first laptop whose price is higher than D dollars:

![nn](img/laptop_budget.png)

In [12]:
class Inventory():
    def __init__(self, csv_filename):
        with open(''.join(['dataset/', csv_filename]), encoding='latin-1') as open_file:
            read_file = csv.reader(open_file)
            rows = list(read_file)
        self.header = rows[0]
        self.rows = rows[1:]
        for row in self.rows:
            row[-1] = float(row[-1])
        self.id_to_row = {}
        for row in self.rows:
            self.id_to_row[row[0]] = row
        self.prices = set()
        for row in self.rows:
            price = float(row[-1])
            self.prices.add(price)
        self.rows_by_price = sorted(self.rows, key=lambda row : row[-1])
            
    def get_laptop_from_id(self, laptop_id):
        for row in self.rows:
            id = row[0]
            if id == laptop_id:
                return row
        return None
    
    def get_laptop_from_id_fast(self, laptop_id):
        if laptop_id in self.id_to_row:
            return self.id_to_row[laptop_id]
        return None
    
    def check_promotion_dollars(self, dollars):
        for row in self.rows:
            price = float(row[-1])
            if price == dollars:
                return True
        for i in range(len(self.rows)):
            for j in range(i, len(self.rows)):
                price1 = float(self.rows[i][-1])
                price2 = float(self.rows[j][-1])
                if price1 + price2 == dollars:
                    return True
        return False

    def check_promotion_dollars_fast(self, dollars):
        if dollars in self.prices:
            return True
        for price1 in self.prices:
            price2 = dollars - price1
            if price2 in self.prices:
                return True
        return False
    
    def find_first_laptop_more_expensive(self, target_price):
        range_start = 0                                   
        range_end = len(self.rows_by_price) - 1                       
        while range_start < range_end:
            range_middle = (range_end + range_start) // 2  
            price = self.rows_by_price[range_middle][-1]
            if price <= target_price:                                                     
                range_start = range_middle + 1             
            else:                                          
                range_end = range_middle
        price = self.rows_by_price[range_start][-1]
        if price <= target_price:                  
            return -1                                      
        return range_start

laptop_inven = Inventory('laptops.csv')
print('$1000:', laptop_inven.find_first_laptop_more_expensive(1000))
print('$10000:', laptop_inven.find_first_laptop_more_expensive(10000))

$1000: 683
$10000: -1


## Conclusion

In this guided project, we've learned that we can answer business questions more efficiently by preprocessing the data. We only explored three possible queries that we might want to do over the data. In general, we often have a lot of different datasets to process and queries to answer. Designing such a class for every type of data in a business and implementing specific query methods takes a lot of time.